<a href="https://colab.research.google.com/github/danielson147/Plate-Number-Classification/blob/master/pnumber_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All packages are imported here

In [0]:

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os, cv2,itertools
import matplotlib.pyplot as plt
%matplotlib inline



Using TensorFlow backend.


This session mount google drive on colab. This will give us access to files on google drive 

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


Data Preparation
Here the following happens
1. Set the path for both image folders directory
2. Set the Rows and Columns to 64 each
3. Set the channel to 3 because the images are in RGB

In [0]:
plate_number_dir = '/gdrive/My Drive/hng6task1/plate_number' #'./plate_number/'
negative_images_dir = '/gdrive/My Drive/hng6task1/negative_images' #'./negative_images/'
ROWS = 64
COLS = 64
CHANNELS = 3


We need to now collect all images in a list




In [0]:
plate_numbers_img = [plate_number_dir+i for i in os.listdir(plate_number_dir)]
negative_images_img = [negative_images_dir+i for i in os.listdir(negative_images_dir)]

We need to now define a function to read and resize all the images

In [0]:
def read_image(file_path):
  img = cv2.imread(file_path, cv2.IMREAD_COLOR)
  resized_img = cv2.resize(img, (ROWS,COLS), interpolation = cv2.INTER_AREA)
  return resized_img

We need to now define a function to prepare the images for usage in our model 

In [0]:
def prep_data(images):
  m = len(images)
  n_x = ROWS*COLS*CHANNELS
  X = np.ndarray((n_x,m), dtype = np.uint8)
  y = np.zeros((1,m))
  print("X.shape is {}".format(X.shape))
  for i,image_file in enumerate(images):
    
    image = read_image(image_file)
    print(i, 'done')
    X[:,i] = np.squeeze(image.reshape((n_x,1)))
    
    if'-' in image_file.lower():
      y[0,i] = 1
      
    elif'glass' in image_file.lower():
      y[0,i] = 0
      
      
    if 1%100 == 0:
      print("Proceed {} of {}".format(i,m))
      
    return X,y

We need to then call the above function and pass it to the two image list

1.   plate_img
2.   negative_img



In [0]:
plate_img, negative_img = prep_data(plate_numbers_img + negative_images_img)

Setting the classes. 1 for plate_number and 0 for negative_images

In [0]:
classes = {0:'Negative_Image', 
           1:'Plate_Number'}

Setting a function for displaying images

In [0]:
def show_images(X, y, idx):
  image = X[idx]
  image = image.reshape((ROWS, COLS, CHANNELS))
  plt.figure(figsize=(4,2))
  plt. imshow(image),
  plt.title(classes[y[idx,0]])
  plt.show()
show_images(plate_img.T, negative_img.T, 0)

Build Logistic Regression Model
Here, we will 
1. import logisticRegressionCV from sklearn
2. Initialise the LRCV
3. Fit our data into the model
4. Print the model accuracy on our training set


In [0]:
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV()
plate_img_lr, neg_img_lr = plate_img.T, negative_img.T.ravel()

clf.fit(plate_img, neg_img_lr)
print("Model accuracy: {:.2f}%".format(clf.score(plate_img_lr, neg_img_lr)*100))


A function to display image prediction

In [0]:
def show_image_prediction(X, idx, model):
  image = X[idx].reshape(1,-1)
  image_class = classes[model.predict(image).item()]
  image = image.reshape((ROWS, COLS, CHANNELS))
  plt.figure(figsize = (4,2))
  plt.imshow(image)
  plt.title("Test {} : I think this is {}".format(idx, image_class))
  
  plt.show()

Since we do not have a test set, we predict our model on our training set

In [0]:
plate_img_lr, neg_img_lr = plate_img.T, negative_img.T

for i in np.random.randint(0, len(plate_img_lr), 5):
  show_image_prediction(plate_img_lr, i, clf)

KNN Classification

In [0]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(plate_img_lr, neg_img_lr)

print("Model accuracy: {:.2f}%".format(knn.score(plate_img_lr, neg_img_lr)*100))

From the above score, we can see that KNN performs poorly compared to Logistic Regression

In [0]:
plate_img_lr, neg_img_lr = plate_img.T, negative_img.T

for i in np.random.randint(0, len(plate_img_lr), 5):
  show_image_prediction(plate_img_lr, i, knn)

Radius Nearest Neighbor

In [0]:
from sklearn.neighbors import RadiusNeighborsClassifier

rnc = RadiusNeighborsClassifier()
rnc.fit(plate_img_lr, neg_img_lr)

print("Model accuracy: {:.2f}%".format(rnc.score(plate_img_lr, neg_img_lr)*100))

In [0]:
plate_img_lr, neg_img_lr = plate_img.T, negative_img.T

for i in np.random.randint(0, len(plate_img_lr), 5):
  show_image_prediction(plate_img_lr, i, rnc)